In [1]:
!pip install opencv-python
!pip install opencv-python-headless

You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [2]:
pip install -U scikit-image

Requirement already up-to-date: scikit-image in /usr/local/lib/python3.6/dist-packages (0.17.2)
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [3]:
import scipy

In [4]:
import tensorflow as tf
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.models import Model
from keras import layers, Sequential
from matplotlib import pyplot
from math import floor
import numpy as np
import cv2
from os import listdir
from os.path import isfile, join
from numpy import expand_dims, array, exp, max
import json
import skimage.feature as feature
from skimage.io import imread
from skimage.color import rgb2gray
from skimage.transform import resize

In [5]:
model1 = VGG16()
model2 = Model(inputs=model1.inputs, outputs=model1.layers[1].output)
# model.add(layers.MaxPool2D(pool_size=(2, 2),strides=(1, 1), padding='valid'))
model = Sequential(layers=model2.layers)
model.add(layers.MaxPool2D(pool_size=(3, 3), strides=(3, 3), padding='same'))
# model = Model(inputs=model.inputs, outputs = model.layers[1].output)

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 75, 75, 64)        0         
Total params: 1,792
Trainable params: 1,792
Non-trainable params: 0
_________________________________________________________________


In [7]:
###########################THIS CODE CHECKS IMAGES FROM A FOLDER#################################
def sortTrainImages():
    # get the class labels from training datasets
    p = {}
    data_path = '../Caltech256/caltech256data/256_ObjectCategories'
    img_gen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1. / 255, rotation_range=20)
    labels = img_gen.flow_from_directory(data_path)
    train_labels = labels.class_indices.keys()

    for lbl in train_labels:
        resnet_path = data_path +'/' + lbl
        p_files = [f for f in listdir(resnet_path) if isfile(join(resnet_path, f))]
        pests_images = np.empty(len(p_files), dtype=object)

        for m in range(0, len(p_files)):
            imageName_x = p_files[m]
            p.update({lbl + "_" + str(m): imageName_x})
            #print(join(resnet_path, p_files[m]))
            ###if m < 50:                
                # get the name of the image
                ##imageName_x = p_files[m]
                ###p.update({lbl + "_" + str(m): imageName_x})

    return p


images_collection = sortTrainImages()

Found 30607 images belonging to 257 classes.


In [8]:
def softMax(temperature, features_list):
    feat_list = [x / temperature for x in features_list]
    feature_list = exp(feat_list - max(feat_list))
    return feature_list / feature_list.sum(axis=0)

In [ ]:
features_prob_dist_array = []

for o, j in images_collection.items():
    
    image_features_array = []
    
    o = o.split('_')
    o = o[0]
    #print("class:", o)
    image_string = '../Caltech256/caltech256data/256_ObjectCategories/' + o + '/' + j
    img = imread(image_string)
    img = resize(img,(224,224,3))
    # get the path of image to extract the class
    img_class = o
    img = img_to_array(img)    
    # expand the image dimensions
    img = expand_dims(img, axis=0)
    print("img:")
    # scale the pixels
    img = preprocess_input(img)
    # get the features of the first layer
    feature_maps = model.predict(img)
    
    numerator = []
    
   
    ix = 1
    square = 3
    for _ in range(square):
        for _ in range(square):
            ax = pyplot.subplot(square, square, ix)
            ax.set_xticks([])
            ax.set_yticks([])
            ##################FLATTEN THE FEATURE for dimensionality reduction##################################
            feature_items = array(feature_maps[0, :, :, ix - 1])
            #convert the floats in the feature into unsigned integers
            features_x = feature_items.astype(np.uint8)
            #get the grey-level cooccurrence matrix
            graycom = feature.greycomatrix(features_x, [1], [0, np.pi/4, np.pi/2, 3*np.pi/4], levels=256)            
            # Find the GLCM properties
            contrast = feature.greycoprops(graycom, 'contrast')
            dissimilarity = feature.greycoprops(graycom, 'dissimilarity')
            homogeneity = feature.greycoprops(graycom, 'homogeneity')
            energy = feature.greycoprops(graycom, 'energy')
            correlation = feature.greycoprops(graycom, 'correlation')            
            ASM = feature.greycoprops(graycom, 'ASM')
            #get the probability distribution for this feature using correlation
            correlation_prob =softMax(0.5,correlation[0])
            #get the conflated distribution for this feature
            conf_numerator = np.prod(correlation_prob)
            numerator.append(conf_numerator)
            #image_features_array.append(correlation_prob)
            image_features_array.append(numerator)
            #pyplot.imshow(feature_maps[0, :, :, ix - 1], cmap='gray')
            
            ix += 1
    denominator = np.trapz(numerator,axis=0)
    #conflated distribution for the image becomes
    conflated_dist = numerator/denominator
    new_rec = {j + "_/" + o: conflated_dist.tolist()}
        
    with open("../files/caltech_target_data_correlation_glcm.json", 'r+') as file:
        # First we load existing data into a dict.
        file_data = json.load(file)
        # Join new_data with file_data inside emp_details
        file_data["target_images"].append(new_rec)
        # Sets file's current position at offset.
        file.seek(0)
        # convert back to json.
        json.dump(file_data, file, indent=4)

    ##if they need to be saved in a bigger file
    #features_prob_dist_array.append(conflated_dist)

img:
img:


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:31: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.


img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:


img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:


/usr/local/lib/python3.6/dist-packages/PIL/TiffImagePlugin.py:772: UserWarning: Possibly corrupt EXIF data.  Expecting to read 8 bytes but only got 6. Skipping tag 41486
  "Possibly corrupt EXIF data.  "
/usr/local/lib/python3.6/dist-packages/PIL/TiffImagePlugin.py:772: UserWarning: Possibly corrupt EXIF data.  Expecting to read 8 bytes but only got 0. Skipping tag 41487
  "Possibly corrupt EXIF data.  "


...

img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:


img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:


img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:


img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:


img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
img:
